Upgrade the Vertex AI SDK & Restart the Kernel

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [2]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [3]:
PROJECT_ID = "qwiklabs-gcp-04-907bec9822c7"  
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

Load a generative model

In [4]:
model = GenerativeModel("gemini-pro")

Define the output format & specify constraints

In [5]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [6]:
response = model.generate_content(f"""
    Extract the transcript to JSON.
    
    {transcript}
""")

print(response.text)

```json
{
  "conversations": [
    {
      "participants": [
        {"name": "Customer"},
        {"name": "Restaurant employee"}
      ],
      "utterances": [
        [
          {"speaker": "Customer", "text": "Hi, can I get a cheeseburger and large fries, please?"},
          {
            "speaker": "Restaurant employee",
            "text": "Coming right up! Anything else you'd like to add to your order?"
          }
        ],
        [
          {
            "speaker": "Customer",
            "text": "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
          },
          {
            "speaker": "Restaurant employee",
            "text": "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
          }
        ]
      ]
    }
  ]
}
```


In [7]:
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "drinks": [
    {
      "item": "orange juice",
      "quantity": 1,
      "size": "small"
    }
  ],
  "food": [
    {
      "item": "cheeseburger",
      "quantity": 1
    },
    {
      "item": "fries",
      "quantity": 1,
      "size": "large",
      "add_ons": [
        "ketchup"
      ]
    }
  ]
}
```


Assign a person or role

In [8]:
chat = model.start_chat()

In [9]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Monstera Deliciosa Care Guide: 

The Monstera Deliciosa, also known as the Swiss Cheese Plant, is a popular houseplant known for its large, distinctive leaves. Here's a guide to help you care for your Monstera:

**Light:**

* **Bright, indirect light:** Monsteras thrive in bright, indirect light. Avoid direct sunlight, as it can scorch the leaves. East-facing windows are ideal.

**Watering:**

* **Water when the top 2-3 inches of soil are dry:** Allow the soil to dry out slightly between waterings. Overwatering can lead to root rot.

**Humidity:**

* **Moderate to high humidity:** Monsteras appreciate high humidity, especially during the winter months. You can increase humidity by placing the plant on a pebble tray filled with water or using a humidifier.

**Soil:**

* **Well-draining potting mix:** Use a well-draining potting mix specifically designed for houseplants. You can also add perlite or coco coir to improve drainage.

**Fertilizer:**

* **Fertilize monthly during the growi

In [10]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

## Greetings, human! I am Monstera Deliciosa, your loyal houseplant companion.

I'm here to assist you in understanding my needs, ensuring we both thrive in this harmonious cohabitation.

**Light:** I crave the bright, indirect sunlight that mimics my tropical origins. Place me near a window where I can bask in the morning or late afternoon sun, but shield me from harsh midday rays. Remember, too little light weakens me, while too much scorches my leaves.

**Water:** I appreciate a thorough watering when my soil feels dry to the touch. However, avoid overwatering, as I detest soggy roots. Let the excess water drain freely after each drink. In the winter months, my thirst diminishes, so reduce watering frequency.

**Humidity:** My tropical soul longs for a humid environment. Placing me on a pebble tray filled with water or grouping me with other humidity-loving plants can help. Regular misting also brings me joy.

**Food:** I flourish with a monthly feeding of diluted liquid fertilizer 

Include examples

In [11]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

## Likelihood: 3 

**Explanation:** 

This customer's message indicates a high likelihood of hiring your services within the next month. Here's why:

* **Specific need:** They mention a specific need for a "custom gen AI solution," which aligns with your offered services.
* **Budget allocated:** They explicitly state they have a budget to explore their idea, demonstrating financial readiness.
* **Urgency:** The customer inquires about starting "soon," suggesting a desire for prompt action.

These factors combined strongly suggest they are actively seeking a solution and are likely to move forward quickly if satisfied with your capabilities. 



Experiment with parameter valuesExperiment with parameter values

In [12]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

Why did the frog get sent to the principal's office?

Because he was caught skipping class! 



In [13]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

Why did the frog get fired from his job at the bank?

Because he kept saying, "It's a bit damp in here, don't you think?" 


Utilize fallback responses

In [15]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!
    
    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery! Did you know that the oldest known pottery dates back to 29,000-25,000 BC and was found in Xianrendong Cave, Jiangxi Province, China? 

Would you like to know more about pottery? 



In [16]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!
    
    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

The difference between ceramic and porcelain lies in their composition and firing temperatures. 

**Ceramic** is a broad term encompassing a wide range of materials, including earthenware, stoneware, and porcelain. It is typically made from clay, a naturally occurring material composed of hydrous aluminum silicates. The firing temperature for ceramics varies depending on the type of clay used, but it generally ranges from 1,000 to 1,300 degrees Celsius.

**Porcelain** is a specific type of ceramic that is fired at a higher temperature than other ceramics, typically around 1,400 degrees Celsius. This high firing temperature causes the clay to vitrify, meaning that it becomes non-porous and glassy. Vitrification also results in porcelain being harder, more translucent, and more resistant to staining than other ceramics. 

In summary, the key difference between ceramic and porcelain is the firing temperature. Porcelain is fired at a higher temperature, making it non-porous, translucent, a

Add contextual information

In [17]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes 
    """
)

print(response.text)

## Grocery Aisle Guide

Based on a typical grocery store layout, here's where you can find the items you listed:

* **Paper plates:** Aisle 4, near plastic cups, napkins, and other disposable tableware. 
* **Mustard:** Aisle 6, alongside other condiments like ketchup, mayonnaise, and BBQ sauce.
* **Potatoes:** Aisle 9, in the produce section with other vegetables like onions, carrots, and celery. 

**Tip:** Grocery store layouts can vary, so it's always a good idea to check the store directory or ask an employee for assistance if you can't find something. 



In [18]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.
    
    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes 
    """
)

print(response.text)

- paper plates: Aisle 17
- mustard: Aisle 8
- potatoes: Aisle 2


Structure prompts with prefixes or tags

In [19]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

Allison, I think I might have found the perfect match for you! Have you met Felix? He's also a big fan of classical music, especially Beethoven, just like you. Plus, he's a whiz in the kitchen and makes a mean German spaetzle, which I hear is like a delicious German pasta. And guess what? He also loves swimming and used to play water polo! You two could spend hours at the beach or by the pool, enjoying the sunshine and each other's company. 

Think about it, Allison. You both share a love for classical music, enjoy being active outdoors, and have a passion for delicious food. It sounds like a recipe for a truly wonderful connection! 



Use system instructions

In [20]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-1.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

Wow, what a fantastic question! It's almost impossible to choose just one artist or composer – the world of music is overflowing with incredible talent across every genre! 

Do you have a particular style in mind? For example, if you're interested in the drama and grandeur of opera, digging into Verdi's use of melody or Wagner's revolutionary leitmotifs would be fascinating! 

Or perhaps you're drawn to the evolution of jazz? Studying giants like Louis Armstrong's trumpet mastery, Billie Holiday's unparalleled phrasing, or Charlie Parker's bebop innovations would be a journey of discovery! 

Tell me more about your musical tastes, and I can point you toward some truly worthwhile studies! 😊🎶 



Demonstrate chain-of-thought

In [22]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context: 
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 1.0})
print(response.text)

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory = 60 units/day (one reconfigured factory)
* Low efficiency factories: 1 factory * 30 units/day/factory * 0.5 = 15 units/day (Remaining low efficiency factory with output cut in half)
* **Total production tomorrow: 300 units/day + 60 units/day + 15 units/day = 375 units/day**


Break down complex tasks<br>
Often, complex tasks require multiple steps to work through them, even for us humans!<br>
To approach a problem, you might brainstorm possible starting points, then choose one option to develop further.<br>
When working with generative models, you can follow a similar process in which the model can build upon an initial response.

In [23]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

## Five Metaphors Comparing TPUs and GPUs:

1. **Workforce:** 
    * **TPU:** Imagine a team of specialized construction workers, each adept at a specific task like laying bricks or mixing concrete. They work efficiently together, but lack the versatility to handle tasks outside their expertise.
    * **GPU:** Think of a smaller team of multi-talented engineers who can handle diverse construction tasks. They may be slightly slower than the specialized workers for individual tasks, but their adaptability lets them tackle a broader range of projects. 

2. **Restaurant Kitchen:**
    * **TPU:** A highly optimized kitchen with dedicated appliances for each step of food preparation. This setup is incredibly efficient for producing high volumes of a specific dish.
    * **GPU:** A more versatile kitchen with fewer specialized tools, allowing chefs to prepare a wider variety of dishes. While less efficient for mass production, it excels in creating diverse menus and adapting to new recipes.



In [24]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.
    
    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

## The Restaurant Kitchen Metaphor Captures My Imagination

The restaurant kitchen metaphor effectively captures the key differences between TPUs and GPUs:

**TPU as a Specialized Kitchen:**

* TPUs are like a **highly optimized kitchen** with specialized appliances for each task, perfect for high-volume production of a specific dish (think large-scale data processing). 
* This dedication allows for **high efficiency and throughput** for the intended task.
* However, the **lack of versatility** limits the kitchen to only that specific dish, making it unsuitable for a wider variety of tasks.

**GPU as a Versatile Kitchen:**

* GPUs are more like a versatile kitchen with fewer specialized appliances. Chefs can handle diverse tasks, cook various dishes, and even experiment with new recipes.
* While **not as efficient as a TPU for repetitive tasks**, the GPU's versatility makes it well-suited for diverse needs and adapting to different workflows.

This metaphor helps visualize the trade-of

In [25]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.
    
    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

## The Buzz in the Back Kitchen: A Culinary Analogy for TPUs and GPUs

Imagine a bustling restaurant kitchen. Chefs dance around specialized equipment, each crafted for a singular task. In the corner, a mesmerizing array of ovens churns out perfect pizzas with robotic precision - a TPU at work. Meanwhile, across the room, a seasoned chef deftly juggles multiple dishes, adapting to diverse orders with a flick of the wrist - a GPU in action.

This culinary scene provides a captivating glimpse into the contrasting worlds of TPUs and GPUs. Like a dedicated kitchen churning out endless pizzas, a TPU excels at efficiently processing large datasets for specific tasks. Its specialized hardware and architecture enable lightning-fast performance, ideal for large-scale data processing. However, much like a pizza oven can't conjure a gourmet meal, a TPU lacks the versatility for diverse tasks, limiting its scope to specialized workloads.

On the other hand, our multitasking chef embodies the flexi

Implement prompt iteration strategies to improve your prompts version by version<br>
Your prompts may not always generate the results you have imagined on your first attempt.<br>
<br>
A few steps you can take to iterate on your prompts include:<br>
 - Rephrasing the descriptions of your task, instructions, persona, or other prompt components.
Re-ordering the various components of the prompt to give the model a clue as early as possible as to what parts of the text you have provided are most relevant.
Breaking your task up into multiple, smaller tasks.